In [16]:
import pandas as pd
import json
import datetime

In [55]:
def decode_v(x):
    r = x[1:-1]
    r = r.replace("'",'"')
    s = r.split(': ')[0]+': '
    r = r.split(s)[1:]
    r = [s+i for i in r]
    r = [i[:-2] if i[-2:]==', ' else i for i in r]
    r = [json.loads(i) for i in r]
    return r

def ts2dt(x):
    return datetime.datetime.utcfromtimestamp(x)


def get_vout(block_path):
    d = pd.read_csv(block_path)
    d['vout'] = d['vout'].map(decode_v)

    t = enumerate(d['vout'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)
    
    vout = t['scriptPubKey'].values
    vout = pd.DataFrame.from_records(vout)
    vout = pd.concat([t.iloc[:,:2], vout], axis=1)
    vout = pd.merge(vout,d.reset_index(), how='left', on='index')
    vout['time'] = vout['time'].map(ts2dt)
    vout['date'] = vout['time'].dt.date
    return vout



def get_vin(block_path):
    d = pd.read_csv(block_path)
    d['vin'] = d['vin'].map(decode_v)

    t = enumerate(d['vin'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)

    vin = t['scriptSig'].map(lambda x: x if x==x else {'asm': None, 'hex': None})#.values
    vin = pd.DataFrame.from_records(vin)
    vin = pd.concat([t.iloc[:,:2], vin], axis=1)
    vin = pd.merge(vin,d.reset_index(), how='left', on='index')

    vin['time'] = vin['time'].map(ts2dt)
    vin['date'] = vin['time'].dt.date

    return vin


In [56]:
def filter_tx(tx, check_path):
    check = pd.read_csv(check_path, index_col=0)
    check['ltime'] = pd.to_datetime(check['ltime'])
    check['date'] = check['ltime'].dt.date
    
    tmp = pd.merge(left=tx, right=check, how='inner', on='date')
    
    tmp['delta_time'] = (tmp['time'] - tmp['ltime']) / datetime.timedelta(minutes=1)
    tmp = tmp[tmp['delta_time'].abs()<30]
    tmp = tmp[tmp['delta_time']>0]
    tmp['delta_sum'] = 2*(tmp['value'] - tmp['btcsum']) / (tmp['btcsum'] + tmp['value'])
    tmp = tmp[tmp['delta_sum'] < 0.1]
    tmp = tmp[tmp['delta_sum'] > 0]

    return tmp

In [58]:
current_block_csv = './rpc_block_834577_txs.csv'
check_csv = './check_btc.csv'

filter_tx(get_vout(current_block_csv),check_csv)

,index,value,asm,desc,hex_x,address,type,in_active_chain,txid,hash,...,hex_y,blockhash,confirmations,time,blocktime,date,ltime,btcsum,delta_time,delta_sum
30086,69,0.000447,OP_HASH160 78f607fec1a3974a5adb0587b319acda5c7...,addr(3Cibk7WTryDWNAPDdDRYwPCF6GTQLiHRZb)#vxsapr9t,a91478f607fec1a3974a5adb0587b319acda5c72c18987,3Cibk7WTryDWNAPDdDRYwPCF6GTQLiHRZb,scripthash,True,51b6c9d1f8a5f0713985a2a70a86a68d20233cc9fec37d...,9ad76c168921ad0a7864d03e35bb3af43c4e1132c7e868...,...,020000000001043df3a4e4ffb59ec5bcecf8c54c0132fe...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.011080
33297,79,0.000453,OP_DUP OP_HASH160 2a46b6273dd8aceeee06643c9afb...,addr(14rY6BUNhGFKek5dEzTpi7X2KgWCiocebx)#0nvhhx29,76a9142a46b6273dd8aceeee06643c9afbf33af27688f8...,14rY6BUNhGFKek5dEzTpi7X2KgWCiocebx,pubkeyhash,True,704e5a2192c4d214150bc848cb89ecba7ad8a6930b9d3d...,704e5a2192c4d214150bc848cb89ecba7ad8a6930b9d3d...,...,010000000129dc5cb1c8bc8b9014a26b2c811c2497760c...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.023495
50028,117,0.000473,0 568daefc6315f737fddafcb4fed275195413c29f,addr(bc1q26x6alrrzhmn0lw6lj60a5n4r92p8s5l9p9zd...,0014568daefc6315f737fddafcb4fed275195413c29f,bc1q26x6alrrzhmn0lw6lj60a5n4r92p8s5l9p9zd8,witness_v0_keyhash,True,c9aece470ac1093c6e534df10f17f74b082dddd0ec7aa0...,81b7ecb7a82f0158820903d186cb71536570054fed6fec...,...,010000000001018ae00ff9492bc4fd6c659e2c45380b50...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.066760
54422,136,0.000459,OP_HASH160 267e4648ceef77f9872d6f1d2a5530c4b0c...,addr(35CYsYS4qLL7TXVrS4utzT1XEV1KwpJoh9)#x7g3508n,a914267e4648ceef77f9872d6f1d2a5530c4b0ce174987,35CYsYS4qLL7TXVrS4utzT1XEV1KwpJoh9,scripthash,True,44279177cf9c365d02b3cb60517bacf40b47604d4bc8a8...,f4be0018273a6ae9ab6848dafc064b81918ba254f3cd2f...,...,0100000000010189751b0a6f365b55ae876528408c38b7...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.036476
72336,143,0.000477,0 44bbe4de0d96b86436b89b26dd99e4d6fb6ffa49,addr(bc1qgja7fhsdj6uxgd4cnvndmx0y6makl7jfm626e...,001444bbe4de0d96b86436b89b26dd99e4d6fb6ffa49,bc1qgja7fhsdj6uxgd4cnvndmx0y6makl7jfm626ea,witness_v0_keyhash,True,688a2c98a8597d031d16880ed7d205a6876efadd54fecd...,3be3a86788e17d4d3a4537b380d33cde15d88c33ced968...,...,01000000000101be163ccfe38f58505933c28aa133e641...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.075504
86532,163,0.000472,0 ff4c25b83ccb1fb534127db62bc0e0b00de93676,addr(bc1qlaxztwpuev0m2dqj0kmzhs8qkqx7jdnk74mwn...,0014ff4c25b83ccb1fb534127db62bc0e0b00de93676,bc1qlaxztwpuev0m2dqj0kmzhs8qkqx7jdnk74mwnx,witness_v0_keyhash,True,1317a86b9dd374660ce8f67de69dabc8fcff17b81508ec...,ea5ffbaa08671ca41f4ada142af98fde56ee20be17ed5d...,...,0100000000010337fa472a8d37851e393535d7207f3677...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.064392
105629,221,0.000450,OP_DUP OP_HASH160 2ef263fff7c67426884b54e1d50b...,addr(15HETRN2Um3udKrP19ZLkCfmfespvKFkAQ)#q64hp5ee,76a9142ef263fff7c67426884b54e1d50b747684c69d2e...,15HETRN2Um3udKrP19ZLkCfmfespvKFkAQ,pubkeyhash,True,379fd97c28cfb6c0185b67c5a81f55baa8287a115249e6...,47c80768c3fcd5380978a2e80ed4cfd7decc557d0b4f7c...,...,01000000000101aee68e3865cce152163db15800a3c5f3...,00000000000000000002b3eb358ccf34216f19688ccbb3...,10781,2024-03-13 22:40:42,1710369642,2024-03-13,2024-03-13 22:35:25,0.000442,5.283333,0.016852
133852,304,0.000482,OP_HASH160 eeb8959abba34e178a092c936caf2867da2...,addr(3PTFsM8HX5wGNhFPrum2vUDqZSGfy5dESh)#0d2rzwyw,a914eeb8959abba34e178a092c936caf2867da2743c987,3PTFsM8HX5wGNhFPrum2vUDqZSGfy5dESh,scrip

In [70]:
f = pd.read_csv('./filtered_rpc_vouts2.csv').reset_index()
f['ltime'].unique()

array(['2024-03-13 22:35:25', '2024-03-13 21:50:27',
       '2024-03-13 22:06:34', '2024-03-13 22:08:39',
       '2024-03-14 11:21:29', '2024-03-14 11:36:48',
       '2024-03-16 09:45:33', '2024-03-17 07:52:21'], dtype=object)

In [ ]:
: